In [ ]:
!nvidia-smi

In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
!yolo task=detect mode=train model=E:\projects\FootbalRoboflow\yolo11n.pt data=E:\projects\FootbalRoboflow\football\data.yaml batch=10 epochs=50 imgsz=1280 plots=True

In [ ]:
!yolo task=detect mode=val model=E:\projects\FootbalRoboflow\runs\detect\train\weights\best.pt data=E:\projects\FootbalRoboflow\football\data.yaml imgsz=1280

In [ ]:
!pip install -q supervision

In [ ]:
!gdown -O "0bfacc_0.mp4" "https://drive.google.com/uc?id=12TqauVZ9tLAv8kWxTTBFWtgt2hNQ4_ZF"
!gdown -O "2e57b9_0.mp4" "https://drive.google.com/uc?id=19PGw55V8aA6GZu5-Aac5_9mCy3fNxmEf"
!gdown -O "08fd33_0.mp4" "https://drive.google.com/uc?id=1OG8K6wqUw9t7lp9ms1M48DxRhwTYciK-"
!gdown -O "573e61_0.mp4" "https://drive.google.com/uc?id=1yYPKuXbHsCxqjA9G-S6aeR2Kcnos8RPU"
!gdown -O "121364_0.mp4" "https://drive.google.com/uc?id=1vVwjW1dE1drIdd4ZSILfbCGPD4weoNiu"

In [ ]:
from ultralytics import YOLO

model = YOLO('runs/detect/train/weights/best.pt')

result = model(source='download_3.mp4', show=False, conf=0.4, save=True)

In [ ]:
from ultralytics import YOLO
from tqdm import tqdm
import supervision as sv

# Initialize the YOLO model
yolo_model = YOLO("E:/projects/FootbalRoboflow/runs/detect/train/weights/best.pt")

# Annotators
box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000')
)

SOURCE_VIDEO_PATH = "E:/projects/FootbalRoboflow/download_3.mp4"
TARGET_VIDEO_PATH = "E:/projects/FootbalRoboflow/download_result.mp4"

# Frame generator from supervision
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

with video_sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):
        results = yolo_model(frame, conf=0.4, save=False)

        # Extract detections from results
        detections = results[0]  # Get the first frame's results
        # Create supervision Detections object
        supervision_detections = sv.Detections.from_ultralytics(detections)

        # Create labels


        # Annotate frame
        annotated_frame = frame.copy()
        annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=supervision_detections)
        video_sink.write_frame(annotated_frame)
